### OCR = Text detection + Text recognition  
![ocr](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-6-L-00.max-800x600.png)

문자를 읽는 OCR은 크게 문자의 영역을 검출하는 Text detection 그리고 검출된 영역의 문자를 인식하는 Text recognition으로 구분할 수 있습니다. 

Text detection은 그동안 우리가 다루어 왔던 일반적인 Object detection 태스크를 문자 찾아내기로 확장한 버전이라고 생각할 수도 있겠습니다. 그러나 앞으로 살펴보겠지만, Text detection은 단순히 Object detection기법 뿐 아니라 Segmentation 기법도 동원되며, 문자가 가지는 독특한 특성까지 감안한 형태로 지속적으로 발전해 왔습니다.

Text recognition은 검출된 영역의 문자가 무엇인지를 인식해 내는 과정입니다. 아마도 MNIST처럼 이미지 안에 담겨 있는 문자를 구분해 내는 작업을 떠올리시겠죠? 하지만 Text detection을 통해 검출된 문자 영역 속에는 MNIST 데이터처럼 문자 단위로 잘 분리되어 있는 텍스트 이미지만 담겨 있는 것이 아닙니다. 이를 위해 OCR 분야에서 사용하는 독특한 모델 구조에 대해 이어 다루게 될 것입니다.



### Text detection
- 이미지 내에서 문자를 검출해낼 때엔 검출하기 위한 최소 단위를 정해야합니다.
- 단어 단위의 탐지는 Object detection의 Regression기반의 Detection방법입니다. Anchor를 정의하고 단어의 유무, 그리고 Bounding box의 크기를 추정해   서 단어를 찾아냅니다.
- 글자 단위의 방식은 Bounding box regression을 하는 대신 글자인 영역을 Segmentation하는 방법으로 접근합니다. 그리고 두 가지 방법을 모두 활용한 방법이 있습니다.

#### 1) Regression
[TextBoxes: A Fast Text Detector with a Single Deep Neural Network](https://arxiv.org/pdf/1611.06779.pdf)  
![regress](https://aiffelstaticprd.blob.core.windows.net/media/images/architecture_of_textboxes.max-800x600.png)  
TextBoxes 이전에는 글자 단위로 인식한 후 결합하는 방식을 취해 왔지만 위 논문에서는 딥러닝 기반의 Detection을 이용하여 단어 단위로 인식합니다. 네트워크의 기본 구조는 SSD: single shot multibox detector를 활용한 것을 볼 수 있습니다. 이를 통해서 빠르게 문자영역을 탐지해 낼 수 있었다고 합니다.

일반적으로 단어들은 가로로 길기 때문에 Aspect ratio가 큽니다. 이에 따라 몇가지 변형을 주게 되는데요, 기존의 SSD에서는 Regression을 위한 Convolution layer에서 3x3 크기의 kernel을 갖습니다. 여기서는 긴 단어의 Feature를 활용하기 위해서 1x5로 convolution filter를 정의하여 사용합니다. 그리고 Anchor box또한 1,2,3,5,7로 큰 aspect ratio로 만들고 이를 vertical offset을 적용하여 세로 방향으로 촘촘한 단어의 배열에 대응하도록 했습니다.  
![regress2](https://aiffelstaticprd.blob.core.windows.net/media/original_images/textbox_anchor.png)  
Grid cell의 중앙을 기점으로 생성할 경우를 예로 든 것이 파란색(aspect ratio:1) 그리고 검은색 박스(aspect ratio:1)입니다.

그리고 이를 수직방향으로 옮겨서 촘촘하게 만들어준 것이 빨간색과 녹색입니다. 
수직방향으로 Anchor box의 중앙을 하나 더 둠으로써 세로로 촘촘하게 Anchor box를 배치할 수 있게 됩니다.

#### 2) Segmentation
[PixelLink: Detecting Scene Text via Instance Segmentation](https://arxiv.org/pdf/1801.01315.pdf)   
![pixellink](https://aiffelstaticprd.blob.core.windows.net/media/images/segmentation_map.max-800x600.png)    
문자들은 매우 촘촘히 배열이 되어있으니 글자영역으로 찾아낸 뒤에 이를 분리해내는 작업이나 연결하는 작업을 더해서 원하는 최소단위로 만들어줘야 합니다.  

PixelLink는 Text 영역을 찾아내는 segmentation과 함께 어느 방향으로 연결되는지 같이 학습을 하여 Text 영역간의 분리 및 연결을 할 수 있는 정보를 추가적으로 활용하고 있습니다.  
![pixel2](https://aiffelstaticprd.blob.core.windows.net/media/images/architecture_pixellink.max-800x600.png)   
PixelLink의 전체적인 구조는 U-Net과 유사합니다. 하지만 output으로 총 9가지의 정보를 얻습니다.

위 그림의 녹색 부분이 input과 output을 의미하는데, output 중 하나는 Text/non-text Prediction을 위한 class segmentation map으로 해당 영역이 Text인지 Non-text인지 예측값을 의미하는 2개의 커널을 가집니다. 그리고 나머지 8가지는 글자의 Pixel을 중심으로 인접한 8개의 Pixel에 대한 연결여부를 의미하는 16개의 커널로 이루어진 Link Prediction map입니다.

위 그림을 보면 conv 1X1, 2(16) 형태의 레이어가 U-Net 구조로 연결되어 인접 pixel간 연결 구조가 지속적으로 유지되도록 하는 모델 구조임을 알 수 있습니다.

이를 통해서 인접한 pixel이 중싱 pixel과 단어단위로 연결된 pixel인지 아니면 분리된 pixel인지 알 수 있으므로, 문자영역이 단어단위로 분리된 Instance segmentation이 가능해집니다.

#### 최근 기술 1) CRAFT
[Character Region Awareness for Text Detection](https://arxiv.org/abs/1904.01941)  
![craft](https://aiffelstaticprd.blob.core.windows.net/media/images/craft_affinity_map.max-800x600.png)  
CRAFT는 Character 단위로 문자의 위치를 찾아낸 뒤 이를 연결하는 방식을 Segmentation기반으로 구현한 방법입니다. 문자의 영역을 boundary로 명확히 구분하지 않고 가우시안 분포를 따르는 원형의 score map을 만들어서 배치시키는 방법으로 문자의 영역을 학습합니다.
문자 단위 라벨을 가진 데이터셋이 많지 않기 때문에, 단어 단위의 정보만 있는 데이터셋에 대해 단어의 영역에 Inference를 한 후 얻어진 문자 단위의 위치를 다시 학습에 활용하는 Weakly supervised learning을 활용했습니다.  

#### 최근 기술 2) Pyramid Mask Text Detector
![pmtd](https://aiffelstaticprd.blob.core.windows.net/media/images/PMTD.max-800x600.png)  
PMTD(Pyramid Mask Text Detector)는 Mask-RCNN의 구조를 활용하여 먼저 Text영역을 Region proposal network로 찾아냅니다. 그다음 Box head에서 더 정확하게 regression 및 classification을 하고 Mask head에서 Instance의 Segmentation을 하는 과정을 거칩니다.

PMTD는 여기서 Mask 정보가 부정확한 경우를 반영하기 위해서 Soft-segmentation을 활용합니다. 이전의 Mask-RCNN의 경우 단어 영역이 Box head에 의해 빨간색으로 잡히면 우측 처럼 boundary를 모두 Text 영역으로 잡지만, PMTD는 단어의 사각형 배치 특성을 반영하여 피라미드 형태의 Score map을 활용합니다.



### Text recognition
#### 1) Unsegmented Data란?
- 분리에 드는 비용이 비싸거나 어려워 Segmentation이 되어있지 않은 데이터
  
#### 2) CRNN
- Unsegmented Data들이 가진 주요한 특징 중 하나는 segment되어 있지 않은 하위데이터들끼리 시퀀스(sequence)를 이루고 있다는 점입니다.
- 문자이미지가 있을 때 여기서 정보를 추출하기 위해서는 Feature Extractor가 필요합니다. Feature Extractor로 사용되는 CNN을 기반의 VGG 또는 ResNet과 같은 네트워크로부터 문자의 정보를 가진 Feature를 얻어낼 수 있습니다. 이렇게 추출된 Feature를 Map-To-Sequence를 통해 Sequence형태의 feature로 변환한 후 다양한 길이의 Input을 처리할 수 있는 RNN으로 넣습니다.

- RNN이 Feature로 부터 문자를 인식하기 위해서는 문자 영역처럼 넓은 정보가 필요하기 때문에 LSTM으로 구성합니다. 그리고 앞의 정보뿐만 아니라 뒤의 정보가 필요하기 때문에 이를 Bidirectional로 구성해서 Bidirectional LSTM을 사용했습니다. Bidirectional LSTM으로 step마다 나오는 결과는 Transcription Layer에서 문자로 변환됩니다.
  
#### 3) CTC
![CTC](https://aiffelstaticprd.blob.core.windows.net/media/original_images/ctc.png)  
- CRNN에서는 Unsegmented Data를 위해 CTC(Connectionist Temporal Classification)를 활용합니다.
- CTC는 Unsegmented data와 같이 Input과 Output이 서로 다른 Length의 Sequence가질 때, 이를 Align 없이 활용하는 방법입니다. CTC의 핵심인 모델의 Output에서 우리가 알고 싶어하는 Label sequence의 확률을 구할 수 있는 방법에 대해 알아보겠습니다.
- 중복되는 경우는 Label Encode에서 같은 문자를 구분하기 위한 Blank를 중복된 라벨 사이를 구분하기 위해 넣어줍니다.  
![CTC2](https://aiffelstaticprd.blob.core.windows.net/media/original_images/wbHRk.png) 

- Edit distance는 두 문자열 사이의 유사도를 판별하는 방법입니다. 이를 이용하여 예측된 단어에서 삽입, 삭제, 변경을 통해 얼마나 적은 횟수의 편집으로 정답에 도달할 수 있는지 최소 거리를 측정합니다. 


#### 4) TPS
[Robust Scene Text Recognition With Automatic Rectification](https://arxiv.org/pdf/1603.03915.pdf)  
![tps](https://aiffelstaticprd.blob.core.windows.net/media/images/spn.max-800x600.png)  
Thin plate spline은 control point를 정의하고 해당 point들이 특정 위치로 옮겨졌을 때, 축방향의 변화를 interpolation하여 모든 위치의 변화를 추정해냅니다. 이를 통해서 전체 이미지 pixel의 변화를 control point로 만들어낼 수 있습니다.

이 논문(Robust Scene Text Recognition With Automatic Rectification)에서는 Control point 20개를 미리 정의합니다. 그리고 Spatial Transformer Network를 통해서 Control point가 얼마나 움직여야 하는지 예측하는 네트워크를 아래 그림과 같이 Recognition model 앞단에 붙여 입력이미지를 정방향으로 맞춰줍니다. TPS 연산은 미분가능한 연산이기 때문에 이 모듈을 Recognition model 앞단에 붙여서 학습이 바로 가능합니다.

[Spatial Transformation Network](https://3months.tistory.com/197)

### Text recognition + Attention

#### 1) Attention sequence prediction
- [Robust Scene Text Recognition With Automatic Rectification](https://arxiv.org/pdf/1603.03915.pdf)
- [Focusing Attention: Towards Accurate Text Recognition in Natural Images](https://arxiv.org/pdf/1709.02054.pdf)  
![asp](https://aiffelstaticprd.blob.core.windows.net/media/original_images/rnn_decoder.png)  
Attention기반의 sequence prediction은 문장의 길이를 고정하고 입력되는 Feature에 대한 Attention을 기반으로 해당 글자의 Label을 prediction합니다. RNN으로 Character label을 뽑아낸다고 생각하면 되는데 첫번째 글자에서 입력 feature에 대한 Attention을 기반으로 label을 추정하고, 추정된 label을 다시 입력으로 사용하여 다음 글자를 추정해내는 방식입니다.
이 때 우리가 20글자를 뽑겠다고 정하게 되면 "YOU"같은 경우에는 3글자를 채우고 빈자리가 문제가 되는데요, 이러한 경우를 위해 미리 정해둔 Token을 사용합니다. 이 Token에는 처음에 사용되는 "start" token 그리고 끝에 사용되는 "end" token이 있습니다. 필요에 따라서 예외처리나 공백을 위한 token을 만들어서 사용하기도 합니다.


#### 2) Transformer
- [A Simple and Strong Convolutional-Attention Network for Irregular Text Recognition](https://arxiv.org/pdf/1904.01375.pdf)
- [hulk89님의 논문리뷰](https://hulk89.github.io/machine%20learning/2019/05/15/A-Simple-and-Robust-Convolutional-Attention-Network-For-Irregular-Text-Recognition/)  
![transformer](https://aiffelstaticprd.blob.core.windows.net/media/images/transformer_rec.max-800x600.png)  
Transformer도 Recognition 모델에 활용되기 시작합니다. 다양한 논문에서 시도되고 있지만 A Simple and Strong Convolutional-Attention Network for Irregular Text Recognition가 대표적인 논문입니다.
위 논문에서는 Irregular text를 잘 인식하기 위해서 2d space에 대한 attention을 활용하여 문자를 인식하기 위해 Transformer를 활용합니다.
Transformer는 Query, Key, Value라는 개념을 통해서 Self-Attention을 입력으로부터 만들어냅니다. 이를 통해서 입력에서 중요한 Feature에 대해 Weight를 주게 되죠.  
Attention의 핵심은 Decoder의 현재 포지션에서 중요한 Encoder의 State에 가중치가 높게 매겨진다는 점입니다. 위에서 Attention이 시각화된 이미지를 눈으로 볼 수 있는데요, Decoder의 각 Step에 따라 입력에 대한 Visual Attention이 시각화된 모습입니다.